# 23.운영 환경에서의 구조적 스트리밍

## 23.1 내고장성과 체크포인팅

+ 장애복구를 위해 체크포인팅과 WAL 정보를 저장하는 신뢰도 높은 파일 시스템의 경로를 쿼리에 설정
+ writeStream의 checkpointLocation 옵션 사용(이 정보가 지워지면 처음부터 해야 함)

In [ ]:
val datapath = "../BookSamples/data/activity-data"

val static = spark.read.json(datapath)

val streaming = spark
    .readStream
    .schema(static.schema)
    .option("maxFilesPerTrigger", 10)
    .json(datapath)
    .groupBy("gt")
    .count()

val query = streaming
    .writeStream
    .outputMode("complete")
    .option("checkpointLocation", "./checkpoint") // check point location
    .queryName("test_stream")
    .format("memory")
    .start()

## 23.2 애플리케이션 변경하기

### 23.2.1 스트리밍 애플리케이션 코드 업데이트하기

+ 사용자 정의 함수는 시그니처가 같은 경우에만 코드 변경
    + 구조적 스트리밍을 사용 중이라면 새로운 버전의 파싱 함수를 만들어 애플리케이션을 다시 컴파일
    + 중대한 변화에는 신규 체크포인트 디렉터리를 지정해야 함
        + 새로운 집계 키 추가, 완전한 쿼리 변경 등        

### 23.2.2 스파크 버전 업데이트하기
상위 호완성을 지키려 하지만 릴리즈 노트를 확인하고 지원하지 않는 경우 새로운 체크포인트 디텍터리를 지정해 애플리케이션을 다시 시작

### 23.2.3 애플리케이션의 초기 규모 산정과 재조정하기
+ 전반적으로 유입률이 처리율보다 크다면 클러스터나 애플리케이션의 크기를 늘려야 함
+ 익스큐터를 제거하거나 애플리케이션에 설정된 자원을 줄인 다음 재시작하는 방식으로 크기를 줄일 수 있음
+ partition 숫자 등 새로운 설정을 적용하는 경우 다시 시작이 필요할 수 있음

## 23.3 메트릭과 모니터링

+ 가장 기본적인 접근

```
streamingQuery.status

{
  "message" : "Waiting for data to arrive",
  "isDataAvailable" : false,
  "isTriggerActive" : false
}
```

+ 최근 현황

```
Array({
  "id" : "47ed4ee9-e12e-467f-9b29-dfe05abf007f",
  "runId" : "4798f354-6c9f-4095-a2cd-659aa9d01d76",
  "name" : "count_based_device",
  "timestamp" : "2020-06-27T00:41:50.338Z",
  "batchId" : 0,
  "numInputRows" : 780119,
  "processedRowsPerSecond" : 131932.8598004397,
  "durationMs" : {
    "addBatch" : 5383,
    "getBatch" : 135,
    "getOffset" : 84,
    "queryPlanning" : 210,
    "triggerExecution" : 5912,
    "walCommit" : 45
  },
  "eventTime" : {
    "avg" : "2015-02-24T03:00:44.892Z",
    "max" : "2015-02-24T15:21:45.941Z",
    "min" : "2015-02-22T00:41:40.120Z",
    "watermark" : "1970-01-01T00:00:00.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 9,
    "numRowsUpdated" : 9,
    "memoryUsedB...
```

+ 일반적으로 배치 주기, 유입률, 처리율에 대한 시간별 변화를 시각화하는 방식이 유용함

## 23.5 스트리밍 리스터를 사용한 고급 모니터링

```
// 콘솔에 출력
spark.streams.addListener(new StreamingQueryListener() {
    override def onQueryStarted(queryStarted: QueryStartedEvent): Unit = {
        println("Query started: " + queryStarted.id)
    }
    override def onQueryTerminated(
      queryTerminated: QueryTerminatedEvent): Unit = {
        println("Query terminated: " + queryTerminated.id)
    }
    override def onQueryProgress(queryProgress: QueryProgressEvent): Unit = {
        println("Query made progress: " + queryProgress.progress)
    }
})


// 카프카를 활용하는 방법
class KafkaMetrics(servers: String) extends StreamingQueryListener {
  val kafkaProperties = new Properties()
  kafkaProperties.put(
    "bootstrap.servers",
    servers)
  kafkaProperties.put(
    "key.serializer",
    "kafkashaded.org.apache.kafka.common.serialization.StringSerializer")
  kafkaProperties.put(
    "value.serializer",
    "kafkashaded.org.apache.kafka.common.serialization.StringSerializer")

  val producer = new KafkaProducer[String, String](kafkaProperties)

  import org.apache.spark.sql.streaming.StreamingQueryListener
  import org.apache.kafka.clients.producer.KafkaProducer

  override def onQueryProgress(event:
    StreamingQueryListener.QueryProgressEvent): Unit = {
    producer.send(new ProducerRecord("streaming-metrics",
      event.progress.json))
  }
  override def onQueryStarted(event:
    StreamingQueryListener.QueryStartedEvent): Unit = {}
  override def onQueryTerminated(event:
    StreamingQueryListener.QueryTerminatedEvent): Unit = {}
}

```